## 사전 작업

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import matplotlib.font_manager as fm
# 설치된 폰트 중 'NanumGothic' 찾기
for font in fm.fontManager.ttflist:
    if 'NanumGothic' in font.name:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['axes.unicode_minus'] = False

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import numpy as np

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### 데이터 로드

In [7]:
df_symptoms = pd.read_csv("./csv/djkim_helpline_Symptom.csv")
df_symptoms[:2]

,_id,disease_korean_title,related_diseases,diseases_symptoms,cause_diseases
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손
1,65dc451dda45daab46acd346,7번 염색체 장완 36 부분의 미세결손 증후군,미세결실 증후군\n(microdeletion syndrome),"발달 지연, 안면 기형",7번 염색체 장완 유전자 말단 부위의 미세 결실


In [8]:
df_symptoms['symptoms'] = df_symptoms['diseases_symptoms'].str.split(',')

In [9]:
df_symptoms = df_symptoms.explode('symptoms')
df_symptoms[:10]

,_id,disease_korean_title,related_diseases,diseases_symptoms,cause_diseases,symptoms
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,중증 정신지체
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,언어발달지연
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,상동행동
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,뇌전증
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,뇌기형
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,특징적인 생김새
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,\n근긴장저하
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,운동발달지연
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,자폐증상
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,발가락 합지증


### 질환의 증상에 대한 줄바꿈 기호 전처리

In [10]:
import re
df_symptoms['symptoms'] = df_symptoms['symptoms'].astype(str) 
patterns = '(|\n|\\\\n|)'
series = []
for contents in df_symptoms['symptoms']:
    text_regex = re.sub(pattern=patterns,repl='',string=contents)
    series.append(text_regex)
series[:2]

['중증 정신지체', ' 언어발달지연']

In [11]:
df_symptoms['symptoms'] = series
df_symptoms[:10]

,_id,disease_korean_title,related_diseases,diseases_symptoms,cause_diseases,symptoms
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,중증 정신지체
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,언어발달지연
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,상동행동
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,뇌전증
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,뇌기형
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,특징적인 생김새
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,근긴장저하
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,운동발달지연
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,자폐증상
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손,발가락 합지증


### 전문 용어의 특수성을 감안하여 모델을 분리하여 토크나이징 착수

In [12]:
df1 = df_symptoms[['disease_korean_title', 'symptoms']].copy()
df2 = df1.copy()

In [13]:
df1 = df1.reset_index(drop=True)

In [14]:
df1[:5]

,disease_korean_title,symptoms
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,중증 정신지체
1,5번 염색체 장완 14.3 부분의 미세결손 증후군,언어발달지연
2,5번 염색체 장완 14.3 부분의 미세결손 증후군,상동행동
3,5번 염색체 장완 14.3 부분의 미세결손 증후군,뇌전증
4,5번 염색체 장완 14.3 부분의 미세결손 증후군,뇌기형


### 2번 데이터프레임에 대해 전처리

In [15]:
# 조사, 어미, 구두점 등 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [16]:
df1.loc[:,'okt_symptoms']= df1['symptoms'].apply(okt_clean)
df1[:2]

,disease_korean_title,symptoms,okt_symptoms
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,중증 정신지체,중증 정신지체
1,5번 염색체 장완 14.3 부분의 미세결손 증후군,언어발달지연,언어 발달 지연


In [17]:
df1[:5]

,disease_korean_title,symptoms,okt_symptoms
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,중증 정신지체,중증 정신지체
1,5번 염색체 장완 14.3 부분의 미세결손 증후군,언어발달지연,언어 발달 지연
2,5번 염색체 장완 14.3 부분의 미세결손 증후군,상동행동,상동 행동
3,5번 염색체 장완 14.3 부분의 미세결손 증후군,뇌전증,뇌전증
4,5번 염색체 장완 14.3 부분의 미세결손 증후군,뇌기형,뇌 기형


### Tokenizing

In [18]:
#ngram_range=(2, 2), analyzer='char'
vectorizer = TfidfVectorizer()
# 'disease_korean_title' 열에 대해 TF-IDF를 적용합니다.
tfidf_matrix_title = vectorizer.fit_transform(df1['disease_korean_title'])
# 'symptoms' 열에 대해 TF-IDF를 적용합니다.
tfidf_matrix_symptoms = vectorizer.fit_transform(df1['okt_symptoms'])

In [19]:
# 라벨 인코딩을 위해 'disease_korean_title'을 숫자로 변환합니다.
df1['disease_label'] = df1['disease_korean_title'].astype('category').cat.codes

In [20]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_symptoms, df1['disease_label'], test_size=0.2, random_state=42)

In [21]:
# 랜덤 포레스트 분류기를 사용하여 모델을 학습합니다.
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
# 테스트 데이터에 대해 예측을 수행하고 정확도를 계산합니다.
y_pred = clf.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.1211022480058013


In [23]:
from sklearn.metrics import f1_score

# F1 스코어를 계산합니다.
f1 = f1_score(y_test, y_pred, average='weighted')

print('F1 Score: ', f1)

F1 Score:  0.08889180130532895


In [24]:
# from sklearn.preprocessing import LabelEncoder
# # LabelEncoder를 생성하고, 'disease_korean_title'을 라벨로 인코딩합니다.
# le = LabelEncoder()
# df1['disease_label'] = le.fit_transform(df1['disease_korean_title'])

In [44]:
def predict_disease():
    # 사용자로부터 증상을 입력받습니다.
    symptom_text = input("증상을 입력하세요: ")
    # 증상 텍스트를 TF-IDF 벡터로 변환합니다.
    tfidf_vector = vectorizer.transform([symptom_text])
    # 각 질병에 대한 증상 벡터와의 코사인 유사도를 계산합니다.
    cosine_similarities = cosine_similarity(tfidf_vector, tfidf_matrix_symptoms)
    # 유사도와 질병 인덱스를 함께 저장합니다.
    disease_similarities = list(zip(cosine_similarities[0], range(len(df1))))
    # 유사도가 90% 이상인 질병을 찾습니다.
    similar_diseases = [index for similarity, index in disease_similarities if similarity > 0.7]
    # 유사도가 높은 순서대로 정렬합니다.
    similar_diseases.sort(key=lambda x: -disease_similarities[x][0])
    # 유사도가 높은 상위 질병의 이름을 반환합니다.
    disease_names = [df1['disease_korean_title'].iloc[i] for i in similar_diseases]
    # 중복된 질병 이름을 제거합니다.
    unique_disease_names = list(set(disease_names))
    # 유사도가 높은 상위 질병을 반환합니다. 반환할 질병의 개수는 100개 또는 가능한 개수 중 작은 값입니다.
    return unique_disease_names[:min(100, len(unique_disease_names))]
# 예측 함수를 호출하여 새로운 증상에 대한 질병을 예측하고 출력합니다.
print(predict_disease())

증상을 입력하세요:  손상


['첨두유합지증', '멜라스증후군', '색소망막염', '불완전골형성', '골취약증', '선천성 풍진증후군', '거짓 부갑상선기능저하증']


In [26]:
df1[:2]

,disease_korean_title,symptoms,okt_symptoms,disease_label
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,중증 정신지체,중증 정신지체,83
1,5번 염색체 장완 14.3 부분의 미세결손 증후군,언어발달지연,언어 발달 지연,83


In [27]:
df1['okt_symptoms'].values

array(['중증 정신지체', '언어 발달 지연', '상동 행동', ..., '만성 용혈 성 빈혈', '신경 기능 학적 이상',
       '만성 용혈 성 빈혈'], dtype=object)

In [45]:
import pickle

In [46]:
with open('search_symptoms.pkl','wb') as pickle_file :
  pickle.dump(obj=clf, file=pickle_file)